In [8]:
# LOW_PRB

"""
PER : (시가총액) / (당기 순이익)
"""

'\nPER : (시가총액) / (당기 순이익)\n'

In [9]:
# imports

import kquant as kq
import datetime as dt


# TMP

import random

In [10]:
# API_account
def set_api_account():
    kq.set_api("KRX2308020", "EQDkUcyI3dK6oIAXqAR8BXOK4bKxHHmH")
    return None

In [11]:
set_api_account()

In [12]:
# LOAD_DATA
# class DATA_LOADER:
class SYMBOL_LOADER:
    @staticmethod
    def load_symbols_df():
        symbols_df = kq.symbol_stock()
        return symbols_df

    class SYMBOL_FILTER:
        @staticmethod
        def filter__market(symbols_df):
            filtered_symbols_df = symbols_df[
                (symbols_df["MARKET"].isin(["코스닥", "유가증권"]))
            ].copy()
            return filtered_symbols_df

        @staticmethod
        def filter__admin_issue(symbols_df):
            filtered_symbols_df = symbols_df[(symbols_df["ADMIN_ISSUE"] == 0)].copy()
            return filtered_symbols_df

        @staticmethod
        def filter_sec_type(symbols_df):
            filtered_symbols_df = symbols_df[
                symbols_df["SEC_TYPE"].isin(["ST", "EF", "EN"])
            ].copy()
            return filtered_symbols_df

    def filter_symbols_df(self, symbols_df):
        symbol_filter = self.SYMBOL_FILTER()
        filtered_symbols_df = symbol_filter.filter__market(symbols_df)
        filtered_symbols_df = symbol_filter.filter__admin_issue(filtered_symbols_df)
        filtered_symbols_df = symbol_filter.filter_sec_type(filtered_symbols_df)
        return filtered_symbols_df

    @staticmethod
    def get_symbols(symbols_df):
        symbols = sorted(set(symbols_df["SYMBOL"]))
        return symbols

    # SYMBOL_LOADER PIPELINE
    def __call__(self):
        symbols_df = self.load_symbols_df()
        filtered_symbols_df = self.filter_symbols_df(symbols_df)
        symbols = self.get_symbols(filtered_symbols_df)
        return symbols

In [ ]:
symbol_loader = SYMBOL_LOADER()
total_symbols = symbol_loader()

In [13]:
class FUNDAMENTAL_LOADER:
    def __init__(self, symbol, date) -> None:
        self.symbol = symbol
        self.date = date
        self.daily_stock_df = kq.daily_stock(
            symbol,
            start_date=date - dt.timedelta(days=7),
            end_date=date,
        )

    def load_recent_close(self):
        daily_stock_df = self.daily_stock_df
        _close = daily_stock_df.sort_values("DATE").tail(1)["CLOSE"].values[0]
        return _close

    def load_recent_marketcap(self):
        daily_stock_df = self.daily_stock_df
        _marketcap = daily_stock_df.sort_values("DATE").tail(1)["MARKETCAP"].values[0]
        return _marketcap

    def load_recent_netprofit(self):
        netprofit_df = kq.account_history(self.symbol, "122700")
        netprofit_df.sort_values("YEARMONTH", inplace=True)
        _netprofit = netprofit_df.tail(1)["VALUE"].values[0]
        return _netprofit

    def load_recent_capital(self):
        capital_df = kq.account_history(self.symbol, "115000")
        capital_df.sort_values("YEARMONTH", inplace=True)
        _capital = capital_df.tail(1)["VALUE"].values[0]
        return _capital

    def __call__(self):
        _close = self.load_recent_close()
        _marketcap = self.load_recent_marketcap()
        _netprofit = self.load_recent_netprofit()
        _capital = self.load_recent_capital()
        return {
            "symbol": self.symbol,
            "close": _close,
            "marketcap": _marketcap,
            "netprofit": _netprofit,
            "capital": _capital,
        }

In [ ]:
symbol_loader = SYMBOL_LOADER()
total_symbols = symbol_loader()

In [ ]:
# tmp
total_symbols = random.sample(total_symbols, 50)
date = dt.date(2023, 1, 5)

In [ ]:
fundamental_data_results = list()
for symbol in total_symbols:
    try:
        fundamental_loader = FUNDAMENTAL_LOADER(symbol=symbol, date=date)
        _fundamental_data = fundamental_loader()
        fundamental_data_results.append(_fundamental_data)
    except:
        pass

In [ ]:
fundamental_df = pd.DataFrame(fundamental_data_results)

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
low_pbr_df = fundamental_df.nsmallest(5, "pbr")

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
low_pbr_df = fundamental_df.nsmallest(5, "pbr")
low_pbr_df["pbr_weight"] = low_pbr_df["pbr"].sum() / low_pbr_df["pbr"]

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
low_pbr_df = fundamental_df.nsmallest(5, "pbr")
low_pbr_df["pbr_weight"] = low_pbr_df["pbr"].sum() / low_pbr_df["pbr"]
low_pbr_df["price_invest"] = (
    low_pbr_df["pbr_weight"] / low_pbr_df["pbr_weight"].sum()
) * 10000000

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
low_pbr_df = fundamental_df.nsmallest(5, "pbr")
low_pbr_df["pbr_weight"] = low_pbr_df["pbr"].sum() / low_pbr_df["pbr"]
low_pbr_df["price_invest"] = (
    low_pbr_df["pbr_weight"] / low_pbr_df["pbr_weight"].sum()
) * 10000000
low_pbr_df["cnt_invest"] = low_pbr_df["price_invest"] // low_pbr_df["close"]

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
low_pbr_df = fundamental_df.nsmallest(5, "pbr")
low_pbr_df["pbr_weight"] = low_pbr_df["pbr"].sum() / low_pbr_df["pbr"]
low_pbr_df["price_invest"] = (
    low_pbr_df["pbr_weight"] / low_pbr_df["pbr_weight"].sum()
) * 10000000
low_pbr_df["cnt_invest"] = low_pbr_df["price_invest"] // low_pbr_df["close"]
list(low_pbr_df.set_index("symbol")['cnt_invest'].astype(int).to_dict().items())

[('036530', 240),
 ('128820', 469),
 ('127710', 935),
 ('001290', 2227),
 ('005380', 8)]

In [20]:
# Signiture
import pandas as pd
import datetime as dt
import logging


def trade_func(
    date: dt.date,
    dict_df_result: dict[str, pd.DataFrame],
    dict_df_position: dict[str, pd.DataFrame],
    logger: logging.Logger,
) -> list[tuple[str, int]]:
    daily_invest_money = 20000000

    """LOAD DATA"""

    """ LOAD DATA (SYMBOL) """
    symbol_loader = SYMBOL_LOADER()
    total_symbols = symbol_loader()

    # tmp
    total_symbols = random.sample(total_symbols, 50)

    """ LOAD DATA (FUNDAMENTAL) """
    fundamental_data_results = list()

    for symbol in total_symbols:
        try:
            fundamental_loader = FUNDAMENTAL_LOADER(symbol=symbol, date=date)
            _fundamental_data = fundamental_loader()
            fundamental_data_results.append(_fundamental_data)
        except:
            pass

    fundamental_df = pd.DataFrame(fundamental_data_results)
    fundamental_df["per"] = fundamental_df["marketcap"] / fundamental_df["netprofit"]
    
    fundamental_df = fundamental_df[fundamental_df["per"] > 0]
    
    low_pbr_df = fundamental_df.nsmallest(5, "per")
    low_pbr_df["per_weight"] = low_pbr_df["per"].sum() / low_pbr_df["per"]
    low_pbr_df["price_invest"] = (
        low_pbr_df["per_weight"] / low_pbr_df["per_weight"].sum()
    ) * daily_invest_money
    low_pbr_df["cnt_invest"] = low_pbr_df["price_invest"] // low_pbr_df["close"]
    symbols_and_orders = list(
        low_pbr_df.set_index("symbol")["cnt_invest"].astype(int).to_dict().items()
    )
    return symbols_and_orders

In [21]:
dict_df_result = kq.backtest_stock_port_daily(
    trade_func,
    "2023-08-01",  # 실제 심사에서는 투자기간 시작일
    "2023-08-30",  # 실제 심사에서는 투자기간 종료일
    init_cash=1_000_000_000,  # 10억원
)

[2023-08-01] 종목: 078930, 주문전 보유수량:      0 주문수량:    182, 매매수량:    182, 주문후 보유수량:    182
[2023-08-01] 종목: 124500, 주문전 보유수량:      0 주문수량:  1,048, 매매수량:  1,048, 주문후 보유수량:  1,048
[2023-08-01] 종목: 031820, 주문전 보유수량:      0 주문수량:  4,247, 매매수량:  4,247, 주문후 보유수량:  4,247
[2023-08-01] 종목: 213500, 주문전 보유수량:      0 주문수량:    270, 매매수량:    270, 주문후 보유수량:    270
[2023-08-01] 종목: 122690, 주문전 보유수량:      0 주문수량:    650, 매매수량:    650, 주문후 보유수량:    650
[2023-08-02] 종목: 013580, 주문전 보유수량:      0 주문수량:    502, 매매수량:    502, 주문후 보유수량:    502
[2023-08-02] 종목: 122690, 주문전 보유수량:    650 주문수량:  1,194, 매매수량:  1,194, 주문후 보유수량:  1,844
[2023-08-02] 종목: 003690, 주문전 보유수량:      0 주문수량:    408, 매매수량:    408, 주문후 보유수량:    408
[2023-08-02] 종목: 356860, 주문전 보유수량:      0 주문수량:    103, 매매수량:    103, 주문후 보유수량:    103
[2023-08-02] 종목: 010690, 주문전 보유수량:      0 주문수량:    126, 매매수량:    126, 주문후 보유수량:    126
[2023-08-03] 종목: 054800, 주문전 보유수량:      0 주문수량:    691, 매매수량:    691, 주문후 보유수량:    691
[2023-08-03] 종목: 002900, 주문전 보유수량:      0 주

In [22]:
total_df = dict_df_result["TOTAL"]

In [23]:
total_df

,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-08-01,TOTAL,0,0,0,0,"6,397",0.0000,0,0,0,"-19,958,906","980,041,094","19,958,906","19,958,906","1,000,000,000",0,0,0,"1,000,000,000",0
1,2023-08-02,TOTAL,0,0,0,0,"8,730",0.0000,0,0,0,"-19,958,070","960,083,024","39,916,976","39,821,156","999,904,180",0,"-95,820","-95,820","1,000,000,000","95,820"
2,2023-08-03,TOTAL,0,0,0,0,"13,173",0.0000,0,0,0,"-19,981,873","940,101,151","59,898,849","59,315,428","999,416,579",0,"-583,421","-583,421","1,000,000,000","583,421"
3,2023-08-04,TOTAL,0,0,0,0,"16,798",0.0000,0,0,0,"-19,980,475","920,120,676","79,879,324","80,265,881","1,000,386,557",0,"386,557","386,557","1,000,386,557",0
4,2023-08-07,TOTAL,0,0,0,0,"21,853",0.0000,0,0,0,"-19,987,995","900,132,681","99,867,319","100,147,255","1,000,279,936",0,"279,936","279,936","1,000,386,557","106,621"
5,2023-08-08,TOTAL,0,0,0,0,"29,311",0.0000,0,0,0,"-19,969,078","880,163,603","119,836,397","118,979,113","999,142,716",0,"-857,284","-857,284","1,000,386,557","1,243,841"
6,2023-08-09,TOTAL,0,0,0,0,"33,323",0.0000,0,0,0,"-19,986,420","860,177,183","139,822,817","140,906,194","1,001,083,377",0,"1,083,377","1,083,377","1,001,083,377",0
7,2023-08-10,TOTAL,0,0,0,0,"35,305",0.0000,0,0,0,"-19,960,750","840,216,433","159,783,567","161,050,999","1,001,267,432",0,"1,267,432","1,267,432","1,001,267,432",0
8,2023-08-11,TOTAL,0,0,0,0,"40,083",0.0000,0,0,0,"-19,933,226","820,283,207","179,716,793","182,314,226","1,002,597,433",0,"2,597,433","2,597,433","1,002,597,433",0
9,2023-08-14,TOTAL,0,0,0,0,"43,938",0.0000,0,0,0,"-19,788,960","800,494,247","199,505,753","199,935,526","1,000,429,773",0,"429,773","429,773","1,002,597,433","2,167,660"
